# Herramienta de explicabilidad: InterpretML

## Introducción

En este notebook, utilizaremos InterpretML para analizar el dataset de _Heart Disease_ y comprender un modelo entrenado en él.

### Utilidad de InterpretML

En el mundo del machine learning, las máquinas aprenden en la oscuridad y los científicos de datos luchan en el vacío para explicar sus decisiones. Sin embargo, gracias a InterpretML, ahora podemos iluminar ese camino. InterpretML es una biblioteca de código abierto que incorpora técnicas de interpretabilidad de vanguardia en un solo lugar. Con esta herramienta, puedes entrenar modelos interpretables de caja de vidrio (glassbox) y explicar sistemas de caja negra (blackbox). InterpretML te ayuda a comprender el comportamiento global de tu modelo o entender las razones detrás de predicciones individuales.

La interpretabilidad es esencial para:

- Depurar modelos.
- Ingeniería de características (formas de mejorar el modelo).
- Detectar problemas de equidad.
- Cooperación humano-AI.
- Cumplimiento normativo.
- Aplicaciones de alto riesgo: Salud, finanzas, judicial, ...

En este notebook, utilizaremos InterpretML para analizar el dataset de Heart Disease y comprender un modelo entrenado en él.

## 1. Preparación del entorno
Aquí instalaremos e importaremos las bibliotecas necesarias. También cargaremos el dataset de Heart Disease.

In [ ]:
# Instalación de interpretML
!pip install interpret

# Importación de bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from interpret import set_visualize_provider
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

# Carga del dataset
df = pd.read_csv('../resources/HeartDiseaseDataset.csv')

## 2. Análisis exploratorio de datos
Antes de entrenar nuestro modelo, es importante comprender los datos. Realizaremos un análisis exploratorio básico para familiarizarnos con el dataset.

Visualización de las primeras filas del dataset:

In [2]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Resumen estadístico del dataset:

In [3]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


Distribución de la variable objetivo:

In [4]:
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

## 3. Preprocesamiento de datos
Antes de entrenar el modelo, necesitamos preprocesar los datos para asegurarnos de que estén en el formato adecuado.

In [5]:
# División del dataset en características (X) y variable objetivo (y)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# División del dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 4. Entrenamiento del modelo
Ahora estamos listos para entrenar nuestro modelo interpretable utilizando la técnica _Explainable Boosting Machine (EBM)_ de InterpretML.

In [6]:
# Entrenamiento del modelo EBM
ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

ExplainableBoostingClassifier()

## 5. Evaluación del modelo
Aquí evaluaremos el rendimiento del modelo utilizando métricas como la precisión y la curva ROC.

In [7]:
# Predicción en el conjunto de prueba
y_pred = ebm.predict(X_test)

# Cálculo de la precisión del modelo
accuracy = (y_pred == y_test).mean()
print("Precisión del modelo:", accuracy)

# Cálculo del área bajo la curva ROC
y_prob = ebm.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print("Área bajo la curva ROC:", auc)

Precisión del modelo: 0.8360655737704918
Área bajo la curva ROC: 0.8976293103448275


## 6. Interpretación del modelo
Una vez que el modelo ha sido entrenado, podemos utilizar las funciones de InterpretML para interpretar y comprender sus predicciones.

### Explicación global
Comencemos examinando el comportamiento global del modelo y las características más importantes.

In [8]:
# Obtener una explicación global del modelo
ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7001/1821558601088/ -->

### 7. Explicación local
También podemos analizar predicciones individuales y comprender las razones detrás de cada una.

In [9]:
# Obtener una explicación local para un conjunto de datos de prueba
ebm_local = ebm.explain_local(X_test, y_test)
show(ebm_local)

<!-- http://127.0.0.1:7001/1821283256256/ -->

## 8. Conclusión
En este notebook, hemos utilizado InterpretML para entrenar un modelo interpretable y comprender su comportamiento en el dataset de Heart Disease. Hemos explorado la explicación global y local del modelo para obtener una comprensión más profunda de sus predicciones de forma intuitiva y visual.